In [9]:
import matplotlib
import matplotlib.pyplot as plt
from utils import *
from scipy.integrate import solve_ivp
from numpy.linalg import inv


In [10]:
############################
# Model parameters
############################
method = "NM_ROM"
nu = 0.001
A = 0.5
filename = "output/burgers_1D/nu_" + str(nu) + "/"


In [11]:
############################
# Read snapshots
############################
print("Reading mesh and solution of the full order model")
mesh, u_ref = read_mesh_and_function(filename + "FOM", "u")


Reading mesh and solution of the full order model


In [16]:
############################
# Perform AutoEncoding
# R: number of dofs in the full order model
# r: number of dofs in the reduced order model
############################

# This is the number of dofs in the reduced order model
r = 82

R, time_steps = u_ref.shape

# split test and train data
X_train, X_test, y_train, y_test = train_test_split(
    u_ref.T, u_ref.T, test_size=0.2, random_state=42)

# This is our input image
input_img = keras.Input(shape=(R, ))

# "encoded" is the encoded representation of the input
# encoded = layers.Dense(r, activation='sigmoid')(input_img)
encoded = layers.Dense(1500, activation='linear')(input_img)
encoded = layers.Dense(r, activation='linear')(encoded)


# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(R, activation='linear')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(r,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

num_epochs = 1

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.fit(X_train, X_train,
                epochs=num_epochs,
                validation_data=(X_test, X_test))

13/13 [==============================] - 0s 12ms/step - loss: 2.0998 - val_loss: 0.7476


In [17]:
############################
# Function space
############################
V = FunctionSpace(mesh, "CG", 1)

############################
# Functions
############################
u = Function(V)
u_old = Function(V)
u0 = Function(V)
v = TestFunction(V)

############################
# Initial condition
############################
u0_expr = Expression(
    "x[0] < 1 ? 1+A*(sin(2*pi*x[0]-pi/2)+1) : 1", degree=1, A=A)
u0.interpolate(u0_expr)


(None, 1001)


In [18]:
u0_red = encoder.predict(u0.vector().get_local().reshape(1, -1))
u0_red = u0_red.reshape(r,)

In [ ]:
import tensorflow as tf

with tf.GradientTape() as g:
  g.watch(x)
  y = x * x
dy_dx = g.gradient(y, x)